# PROJET 5 : Catégorisez automatiquement les questions 

**PLAN DE PROJET**
1. Titre de projet : PROJET 5 - Catégorisez automatiquement les questions


2. Chargement de bibliothèques


3. Récupérer les données + Séparation de données en test et train
    - Enregistrement de fichiers en .csv :
        - X_train.csv
        - y_train.csv
        - X_test.csv
        - y_test.csv


4. Data cleaning
    - Features :
        - Enlever les balises HTML
        - Enlever la ponctuation
        - Mise en minuscule et tokenization
        - Enlever les stopwords
    - Target :
        - Enlever les balises "<>"


5. Feature engineering 
    - Recodage en bigrams
    - Fusion de title, body + bigrams


6. Analyse exploratoire
    - Analyses univariées
        - Description générale : Longueur de posts, nombre de tags
        - Bag of words : Les expressions les plus fréquentes : feature & target
            - Arrays générées:
                - X_train_bow
                - X_train_vocab_bow
                - X_train_dist_bow
                - y_train_bow
                - y_train_vocab_bow
                - y_train_dist_bow
                
                
        - TF - IDF : Les expressions les plus fréquentes : feature & target
             - Arrays générées:
                  - X_train_ifidf
                  - X_train_vocab_ifidf
                  - X_train_dist_ifidf
                  - y_train_ifidf
                  - y_train_vocab_ifidf
                  - y_train_dist_ifidf
                  

    - Analyse multivarié 
    **QUESTION : Peut-on considérer LDA comme analyse multivariée ?**
    
    
    - Réduction de dimensions
    **QUESTION : Peut-on faire un word2vec ?**
    
    
        

# Chargement de bibliothéques

In [1]:
# Import the libraries
import joblib
from IPython.core.display import display, HTML
import numpy as np
import pandas as pd
import random
from pandas import Series, DataFrame
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer 
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE


import nltk
#nltk.download()  # Download text data sets, including stop words
from nltk.corpus import stopwords # Import the stop word list
import re

# Import BeautifulSoup into your workspace
from bs4 import BeautifulSoup 

# Libraries for data visualisation
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

In [2]:
#Disable warning for .loc
pd.options.mode.chained_assignment = None  # default='warn'

# Récupération de données

Les données était récupérées de stack overflow à l'aide de code suivant :

SELECT body, title, tags FROM posts 
WHERE title is not null and body is not null and tags is not null and FavoriteCount>50


In [3]:
data_load = pd.read_csv(
    'Data/data_best_rating50.csv', sep=',')

In [4]:
data_load.shape

(19687, 3)

In [5]:
data_load.head()

,body,title,tags
0,<p>I've been using Eclipse with RDT (not RadRa...,What Ruby IDE do you prefer?,<ruby><ide><editor>
1,<p>We are developing an application that invol...,How to generate sample XML documents from thei...,<xml><xsd><dtd><test-data>
2,<p>I know that IList is the interface and List...,When to use IList and when to use List,<c#><.net>
3,<p>I develop C++ applications in a Linux envir...,What tools do you use to develop C++ applicati...,<c++><linux><eclipse><gdb><valgrind>
4,<p>What would be the most efficient way to com...,What is the most effective way for float and d...,<c++><algorithm><optimization><floating-point>


Exemple de la première question:

In [6]:
print (data_load['body'][2])

<p>I know that IList is the interface and List is the concrete type but I still don't know when to use each one. What I'm doing now is if I don't need the Sort or FindAll methods I use the interface. Am I right? Is there a better way to decide when to use the interface or the concrete type?</p>



## Création d'un jeu de données test

Notre variable cible est la variable tags. Nous avons deux variables texte qui nous permettrons estimer le tag : variables title et body. Nous allons séparer 30 % de données qui seront utilisées plus tard afin de tester les modèles :

In [7]:
target = data_load['tags']

In [8]:
data=data_load.drop(['tags'], axis=1)

In [9]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(data, target, test_size=0.5, random_state=1)

In [10]:
print ("Le jeu de données X_train contient", X_train.shape[0], "observations et", X_train.shape[1], "features.") 
print ("Le vecteur y_train contient", y_train.shape[0], "observations.") 
print ("Le jeu de données X_test contient", X_test.shape[0], "observations et", X_test.shape[1], "features.") 
print ("Le vecteur y_test contient", y_test.shape[0], "observations.")  

Le jeu de données X_train contient 9843 observations et 2 features.
Le vecteur y_train contient 9843 observations.
Le jeu de données X_test contient 9844 observations et 2 features.
Le vecteur y_test contient 9844 observations.


Exporter les jeux de données en .csv

In [11]:
X_train.to_csv('Data/X_train.csv', sep='\t')
X_test.to_csv('Data/X_test.csv', sep='\t')
y_train.to_csv('Data/y_train.csv', sep='\t', header=False)
y_test.to_csv('Data/y_test.csv', sep='\t', header=False)

# Data cleaning 

## Tester les fonctions sur un exemple de features

### Enlever les balises html

Pour enlever les balises HTML, nous allons utiliser le package BeautifulSoup:

In [12]:
# Initialize the BeautifulSoup object on a single movie review     
example1 = BeautifulSoup(X_train['body'][1])  

# Print the raw review and then the output of get_text(), for 
# comparison
print (X_train['body'][1])
print (example1.get_text())

<p>We are developing an application that involves a substantial amount of XML transformations. We do not have any proper input test data per se, only DTD or XSD files. We'd like to generate our test data ourselves from these files. Is there an easy/free way to do that?</p>

<p><strong>Edit</strong></p>

<p>There are apparently no free tools for this, and I agree that OxygenXML is one of the best tools for this.</p>

We are developing an application that involves a substantial amount of XML transformations. We do not have any proper input test data per se, only DTD or XSD files. We'd like to generate our test data ourselves from these files. Is there an easy/free way to do that?
Edit
There are apparently no free tools for this, and I agree that OxygenXML is one of the best tools for this.



### Enlever la ponctuation

Nous allons enlever la ponctuation tout en gardant les nombres, lettres ainsi que les signes "-", "+" et "#" étant donné que ces signes sont souvent utilisés dans l'informatique. 

In [13]:
letters_only = re.sub("[^#+a-zA-Z]",       # The pattern to search for
                      " ",                   # The pattern to replace it with
                      example1.get_text() )  # The text to search
print (letters_only)

We are developing an application that involves a substantial amount of XML transformations  We do not have any proper input test data per se  only DTD or XSD files  We d like to generate our test data ourselves from these files  Is there an easy free way to do that  Edit There are apparently no free tools for this  and I agree that OxygenXML is one of the best tools for this  


### Mise en minuscule et tokenization

In [14]:
lower_case = letters_only.lower()        # Convert to lower case
words = lower_case.split()               # Split into words

### Stopwords

In [15]:
# Let's have a look to the stopwords from ntlk.corpus
print (stopwords.words("english")) 

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [16]:
# Remove stop words from "words"
words = [w for w in words if not w in stopwords.words("english")]
print (words)

['developing', 'application', 'involves', 'substantial', 'amount', 'xml', 'transformations', 'proper', 'input', 'test', 'data', 'per', 'se', 'dtd', 'xsd', 'files', 'like', 'generate', 'test', 'data', 'files', 'easy', 'free', 'way', 'edit', 'apparently', 'free', 'tools', 'agree', 'oxygenxml', 'one', 'best', 'tools']


### Lemmatisation

In [17]:
lemmatizer = WordNetLemmatizer() 

lems = []

for word in words:
    word = lemmatizer.lemmatize(word)
    lems.append(word)
    
print(lems)

['developing', 'application', 'involves', 'substantial', 'amount', 'xml', 'transformation', 'proper', 'input', 'test', 'data', 'per', 'se', 'dtd', 'xsd', 'file', 'like', 'generate', 'test', 'data', 'file', 'easy', 'free', 'way', 'edit', 'apparently', 'free', 'tool', 'agree', 'oxygenxml', 'one', 'best', 'tool']


## Tester les fonctions sur un exemple de targets

Les tags (notre variable cible) ont une forme spécifiques. Les mots sont entre "<>". Les tags sont parfois composés par deux, voir plusieurs mots qui sont séparés par "-". Nous allons enlever les balises.

In [18]:
#Take the first tag from the dataset
tag = y_train[1]

In [19]:
#Check
tag

'<xml><xsd><dtd><test-data>'

In [20]:
#Remove first '<'
tag = tag[1:]

In [21]:
#Check
tag

'xml><xsd><dtd><test-data>'

In [22]:
#Remove last '>'
tag = tag[:-1]

In [23]:
#Check
tag

'xml><xsd><dtd><test-data'

In [24]:
#Remove remaining '><' signs
tag = tag.split('><')

In [25]:
#Check
tag

['xml', 'xsd', 'dtd', 'test-data']

In [26]:
#Converting back to string
tag_clean = " ".join(tag)  

In [27]:
#Check
tag_clean

'xml xsd dtd test-data'

## Pipelines pour data cleaning

### Features

Nous allons utiliser la fonction suivante pour automatiser le processus de data cleaning :

In [28]:
def post_to_words( raw_post ):
    """Function to convert a raw document to a string of words.
    
    Inputs : 
    
    - raw_post : a single string 
    
    Output :
    
    - a single string containing a preprocessed document"""
    
    # 1. Remove HTML
    post_text = BeautifulSoup(raw_post).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^#+a-zA-Z]", " ", post_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Lematize
    lemmatizer = WordNetLemmatizer() 

    lems = []

    for word in meaningful_words:
        word_clean = lemmatizer.lemmatize(word)
        lems.append(word_clean)
    #
    # 7. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( lems )) 

In [29]:
X_train['body_clean']=X_train['body'].apply(lambda x: post_to_words(x))

In [30]:
X_train['title_clean']=X_train['title'].apply(lambda x: post_to_words(x))

In [31]:
X_train.head()

,body,title,body_clean,title_clean
4183,<p>For some reason it looks like constructor d...,How do I create a custom Error in JavaScript?,reason look like constructor delegation work f...,create custom error javascript
13188,<p>How long can latitude and longitude be?</p>...,What is the maximum length of latitude and lon...,long latitude longitude getting long length se...,maximum length latitude longitude
2147,<p>How can I get crash data (stack traces at l...,How do I obtain crash-data from my Android app...,get crash data stack trace least android appli...,obtain crash data android application
7588,<p>I'm wondering what the best way is to have ...,".NET - What's the best way to implement a ""cat...",wondering best way else fails catch mean handl...,net best way implement catch exception handler
12329,<p>I need to parse an RSS feed (XML version 2....,How to parse an RSS feed using JavaScript?,need parse r feed xml version display parsed d...,parse r feed using javascript


Nous allons aussi construire une fonction qui nous servira comme paramètre de tokenizer personnalisé que nous allons appliquer dans l'outil CountVectorizer, afin de garder la structure de bigrams que l'on a créé pendant la transformation de features. Nous allons séparer les mot tout simplement en utilisant méthode .split() qui sépare les mots par espace et laisse les caractères spéciaux comme par exemple '#' collé sur les mots, contrairement à la méthode .tokenize() 

In [32]:
def my_tokenizer(doc):
    
    """Function defining personalized tokenizer for sklearn's CountVectorizer in order to keep the created 
    bigrams in tag's format.
    
    Input:
    
    - doc : string to be tokenized
    
    Output:
    
    - tokenized string
    """
    
    
    tokens = doc.split()
    
    return tokens

Nous allons définir les stop-words que nous n'avons pas enlevé dans l'étape de cleaning, mais qui figurent souvent dans les posts et qui n'ont pas de valeur informative. La liste sera également utilée en paramètre de CountVectorizer et TfidfVectorizer.

In [33]:
stop_words = ["i'm", 'would', '1', '0', '2', "i've", 'could', 'anyone', 'also', '3', 'thanks', 
               'two', 'however', "i'd", '5', "+", "#", "im", "ive", "dont", "cant", "id", ")", "("]

### Target

In [34]:
def cleaning_target(raw_target):
    """Function to remove '<>' signs from target list and to replace them with a space
    
    Arguments :
    - raw_target : a Series of tags
    
    Return :
    - Series with cleaned tags
       
    """
    
    #Remove first '<'
    tag = raw_target[1:]
    
    #Remove last '>'
    tag = tag[:-1]
    
    #Remove remaining '><' signs
    tag = tag.split('><')
    
    #Converting back to string
    tag_clean = " ".join(tag)  
    
    return tag_clean

In [35]:
y_train_clean=y_train.apply(lambda x: cleaning_target(x))

In [36]:
y_train_clean.head()

4183                     javascript exception
13188      database-design latitude-longitude
2147                android crash stack-trace
7588     c# .net exception exception-handling
12329                     javascript html rss
Name: tags, dtype: object

# Feature engineering

## Créer les bigrams

Nous allons créer des bigrams, car les tags sont souvent composés par deux mots, séparés par "-". Les bigrams créé à partir de text de body et title vont donc être mis au même format que les tags.

### Tester les fonctions sur un exemple de post

In [37]:
#Take the first element of the cleaned body text
body = X_train['body_clean'][1]
body

'developing application involves substantial amount xml transformation proper input test data per se dtd xsd file like generate test data file easy free way edit apparently free tool agree oxygenxml one best tool'

In [38]:
#Split it to a list of words
body = body.split()

In [39]:
#Check
body[:10]

['developing',
 'application',
 'involves',
 'substantial',
 'amount',
 'xml',
 'transformation',
 'proper',
 'input',
 'test']

In [40]:
#Creating bigrams object
bigram = nltk.bigrams(body)

In [41]:
#Creating a list of bigram objects
liste = list(bigram)

In [42]:
#Check
liste[:10]

[('developing', 'application'),
 ('application', 'involves'),
 ('involves', 'substantial'),
 ('substantial', 'amount'),
 ('amount', 'xml'),
 ('xml', 'transformation'),
 ('transformation', 'proper'),
 ('proper', 'input'),
 ('input', 'test'),
 ('test', 'data')]

In [43]:
#Creating a tag-like word from tuples
bigrams_rec = []

for i in range(len(liste)):
    tuple1 = liste[i]
    elt = tuple1[0] + '-' + tuple1[1]
    bigrams_rec.append(elt)

In [44]:
#Creating a string from list of tag-like bigrams
bigrams_str = " ".join(bigrams_rec)

### Créer des pipelines pour le recodage en bigrams

Nous allons utiliser deux fonctions afin de :
    1. créer les bigrams
    2. recoder les bigrams dans le même format comme les tags dans notre variable cible

In [45]:
def create_bigrams(var):
    
    """Function which generates a liste of tuples representing bigrams. 
    
    Arguments :
    - var : feature to be recoded
    
    Returns :
    - a Series of recoded feature  
    
    """
   
   #Take the i-th element of the cleaned body text
    body = var    
        
    #Split it to a list of words
    body = body.split()
    
    #Creating bigrams object
    bigram = nltk.bigrams(body)
    
    #Creating a list of bigram objects 
    liste_bigram = list(bigram)
        
        
    return liste_bigram

In [46]:
X_train['body_bigram'] = X_train['body_clean'].apply(lambda x : create_bigrams(x))

In [47]:
X_train['body_bigram'].head()

4183     [(reason, look), (look, like), (like, construc...
13188    [(long, latitude), (latitude, longitude), (lon...
2147     [(get, crash), (crash, data), (data, stack), (...
7588     [(wondering, best), (best, way), (way, else), ...
12329    [(need, parse), (parse, r), (r, feed), (feed, ...
Name: body_bigram, dtype: object

In [48]:
def recode_bigrams(liste):
    """Function which recode the bigrams generated by recode_bigram function.
    Each bigram has the same structure as the stack overflow tages :
    it is composed of two words and the words are separated by '-'  
    
    Arguments :
    - liste : a liste containing bigram tuples
    
    Return :
    
    - a string of recoded bigrams_rec
    """
    
    #Creating a tag-like word from tuples
    bigrams_rec = []

    for i in range(len(liste)):
        tuple1 = liste[i]
        elt = tuple1[0] + '-' + tuple1[1]
        bigrams_rec.append(elt)
        
    return " ".join(bigrams_rec)

In [49]:
X_train['body_bigram_clean'] = X_train['body_bigram'].apply(lambda x : recode_bigrams(x))

In [50]:
X_train['body_bigram_clean'].head()

4183     reason-look look-like like-constructor constru...
13188    long-latitude latitude-longitude longitude-get...
2147     get-crash crash-data data-stack stack-trace tr...
7588     wondering-best best-way way-else else-fails fa...
12329    need-parse parse-r r-feed feed-xml xml-version...
Name: body_bigram_clean, dtype: object

Nous allons appliquer les mâmes fonctions sur la feature title :

In [51]:
X_train['title_bigram'] = X_train['title_clean'].apply(lambda x : create_bigrams(x))

In [52]:
X_train['title_bigram_clean'] = X_train['title_bigram'].apply(lambda x : recode_bigrams(x))

In [53]:
X_train['title_bigram_clean'].head()

4183           create-custom custom-error error-javascript
13188    maximum-length length-latitude latitude-longitude
2147     obtain-crash crash-data data-android android-a...
7588     net-best best-way way-implement implement-catc...
12329           parse-r r-feed feed-using using-javascript
Name: title_bigram_clean, dtype: object

### Fusionner le title, body et les bigrams

Afin de simplifier le travail exploratiore et les recodages de text qui va être utilisé dans la partie analyse, nous allons créer une feature unique composée par le text de title, de body et des bigrams créés à partir de ces deux features :

In [54]:
X_train['post'] = X_train['title_clean'] + " " + X_train['body_clean'] + " " + X_train['body_bigram_clean'] + " " + X_train['title_bigram_clean']  

In [55]:
X_train['post'].head()

4183     create custom error javascript reason look lik...
13188    maximum length latitude longitude long latitud...
2147     obtain crash data android application get cras...
7588     net best way implement catch exception handler...
12329    parse r feed using javascript need parse r fee...
Name: post, dtype: object

In [56]:
X_train['post'][1]

'generate sample xml document dtd xsd developing application involves substantial amount xml transformation proper input test data per se dtd xsd file like generate test data file easy free way edit apparently free tool agree oxygenxml one best tool developing-application application-involves involves-substantial substantial-amount amount-xml xml-transformation transformation-proper proper-input input-test test-data data-per per-se se-dtd dtd-xsd xsd-file file-like like-generate generate-test test-data data-file file-easy easy-free free-way way-edit edit-apparently apparently-free free-tool tool-agree agree-oxygenxml oxygenxml-one one-best best-tool generate-sample sample-xml xml-document document-dtd dtd-xsd'

### Bag of words

Nous allons transformer les données text en matrice creuse, appelée Bag-of-Words, qui indique le nombre d'apparitions d'un mot dans la chaîne de caractères. A l'aide de cette transformation, nous allons pouvoir effectuer des opérations numériques sur les chaînes de caractères, notamment calculer des caractéristiques univariées telles que fréquences.  


La matrice représentant Bag-of-Words contient autant de colonnes que le nombre de mots dans le corpus (ensemble de posts dans notre cas). Nous allons utiliser une option max_features = 50 000 qui limite le nombre de features à 50 000.

#### Feature

In [57]:
#First execution
print ("Creating the bag of words...\n")

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = my_tokenizer,    \
                             preprocessor = None, \
                             strip_accents=None,
                             max_features = 5000,
                             lowercase=False,
                             stop_words = stop_words)                       
                            

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.

X_train_bow = vectorizer.fit_transform(X_train['post'])

# Numpy arrays are easy to work with, so convert the result to an 
# array
X_train_bow = X_train_bow.toarray()

Creating the bag of words...



In [58]:
X_train_bow.shape

(9843, 5000)

In [59]:
# Create a vocabulary of features
X_train_vocab_bow = vectorizer.get_feature_names()

In [60]:
# Sum up the counts of each vocabulary word
X_train_dist_bow = np.sum(X_train_bow, axis=0)

Nous allons afficher 50 mots les plus fréquents :

In [61]:
freq_features_bow = DataFrame(zip(X_train_vocab_bow, X_train_dist_bow))
freq_features_bow.rename({0:'Word', 1:'Frequence'}, axis = 1, inplace = True)
freq_features_bow.sort_values(['Frequence'], inplace = True, ascending=False)
freq_features_bow['Pourcentage'] = round(freq_features_bow['Frequence']*100/sum(freq_features_bow['Frequence']),2)
freq_features_bow[:50]

,Word,Frequence,Pourcentage
1545,file,5163,0.77
4596,use,4466,0.67
4625,using,4357,0.65
2449,like,4116,0.61
161,android,3923,0.59
1753,get,3630,0.54
732,code,3509,0.52
4792,way,3475,0.52
4137,string,3149,0.47
677,class,3143,0.47


Nous constatons que dans les 50 mots les plus courants, nous avons des expressions qui ne nous apporte pas de valeur informative - nous allons donc actualiser la liste de stop words à enlever et réexécuter la transformation en BOW.

In [62]:
#Stop-words update
stop_words += ['using', 'use', 'way', 'want', 'new', 'one', 'work', 'need', 'example', 'know',
               'question', '4', 'line', 'make', '*', 'something', 'find', 'problem', 'following', 
               'create', 'x', 'run', "'", 'trying', 'change', 'used', 'see', 'possible', 'tried', 
               'first', 'without', 'found', 'seems', 'e', 'different', 'b', '8', '10', 'best', 
               'thing', "can't", '6', 'simple', 'answer', '7', 'working', 'good', 'look', 'right', 
               'help', 'another', 'v', 'try', 'point', 'would-like', '0-0', 'default', 'able', 'really', 
               'say', 'looking', 'show', 'idea', 'n', 'even', 'understand', 'better', 'please', 'p',
               'instead', 'think', 'else', 'etc', '$', '9', 'since', 'already', 'give', 'sure']

In [63]:
#Second execution after stop-words update
print ("Creating the bag of words...\n")

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = my_tokenizer,    \
                             preprocessor = None, \
                             strip_accents=None,
                             max_features = 5000,
                             lowercase=False,
                             stop_words = stop_words)                       
                            

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.

X_train_bow = vectorizer.fit_transform(X_train['post'])

# Numpy arrays are easy to work with, so convert the result to an 
# array
X_train_bow = X_train_bow.toarray()

Creating the bag of words...



In [64]:
# Second execution
X_train_bow.shape

(9843, 5000)

In [65]:
# Second execution
# Create a vocabulary of features
X_train_vocab_bow = vectorizer.get_feature_names()

In [66]:
# Second execution
# Sum up the counts of each vocabulary word
X_train_dist_bow = np.sum(X_train_bow, axis=0)

Nous allons afficher 50 mots les plus fréquents :

In [67]:
# Second execution
freq_features_bow = DataFrame(zip(X_train_vocab_bow, X_train_dist_bow))
freq_features_bow.rename({0:'Word', 1:'Frequence'}, axis = 1, inplace = True)
freq_features_bow.sort_values(['Frequence'], inplace = True, ascending=False)
freq_features_bow['Pourcentage'] = round(freq_features_bow['Frequence']*100/sum(freq_features_bow['Frequence']),2)
freq_features_bow[:50]

,Word,Frequence,Pourcentage
1552,file,5163,0.89
2457,like,4116,0.71
162,android,3923,0.67
1756,get,3630,0.62
733,code,3509,0.60
4140,string,3149,0.54
677,class,3143,0.54
1705,function,3019,0.52
1018,data,2827,0.48
2272,java,2683,0.46


#### Titre uniquement

In [68]:
X_train_title_bow = vectorizer.fit_transform(X_train['title_clean'])

# Numpy arrays are easy to work with, so convert the result to an array
X_train_title_bow = X_train_title_bow.toarray()

In [69]:
X_train_title_bow.shape

(9843, 5000)

In [70]:
# Create vocabulary
X_train_title_vocab_bow = vectorizer.get_feature_names()

In [71]:
# Sum up the counts of each vocabulary word
X_train_title_dist_bow = np.sum(X_train_title_bow, axis=0)

In [72]:
freq_features_bow_titre = DataFrame(zip(X_train_title_vocab_bow, X_train_title_dist_bow))
freq_features_bow_titre.rename({0:'Word', 1:'Frequence'}, axis = 1, inplace = True)
freq_features_bow_titre.sort_values(['Frequence'], inplace = True, ascending=False)
freq_features_bow_titre['Pourcentage'] = round(freq_features_bow_titre['Frequence']*100/sum(freq_features_bow_titre['Frequence']),2)
freq_features_bow_titre[:50]

,Word,Frequence,Pourcentage
1624,file,645,1.40
1356,difference,499,1.08
3330,python,457,0.99
180,android,445,0.96
2238,java,421,0.91
1786,git,416,0.90
4077,string,401,0.87
2249,javascript,380,0.82
1747,get,354,0.77
2457,list,242,0.52


#### Cible

In [73]:
print ("Creating the bag of words...\n")

y_train_bow = vectorizer.fit_transform(y_train_clean)

# Numpy arrays are easy to work with, so convert the result to an 
# array
y_train_bow = y_train_bow.toarray()

Creating the bag of words...



In [74]:
y_train_bow.shape

(9843, 4998)

In [75]:
# Creating tag's vocabulary
y_train_vocab_bow = vectorizer.get_feature_names()

In [76]:
# Sum up the counts of each vocabulary word
y_train_dist_bow = np.sum(y_train_bow, axis=0)

In [77]:
freq_target_bow = DataFrame(zip(y_train_vocab_bow, y_train_dist_bow))
freq_target_bow.rename({0:'Word', 1:'Frequence'}, axis = 1, inplace = True)
freq_target_bow.sort_values(['Frequence'], inplace = True, ascending=False)
freq_target_bow['Pourcentage'] = round(freq_target_bow['Frequence']*100/sum(freq_target_bow['Frequence']),2)
freq_target_bow[:50]

,Word,Frequence,Pourcentage
2404,javascript,1039,3.29
3558,python,948,3.01
2377,java,872,2.76
118,android,710,2.25
663,c#,582,1.84
1808,git,524,1.66
668,c++,433,1.37
2305,ios,373,1.18
2442,jquery,353,1.12
2063,html,351,1.11


#### Sauvegarder les données

In [78]:
np.save('Data/X_train_bow', X_train_bow)
np.save('Data/X_train_vocab_bow', X_train_vocab_bow)
np.save('Data/X_train_dist_bow', X_train_dist_bow)
np.save('Data/X_train_title_bow', X_train_title_bow)
np.save('Data/X_train_title_vocab_bow', X_train_title_vocab_bow)
np.save('Data/X_train_title_dist_bow', X_train_title_dist_bow)
np.save('Data/y_train_bow', y_train_bow)
np.save('Data/y_train_vocab_bow', y_train_vocab_bow)
np.save('Data/y_train_dist_bow', y_train_dist_bow)

### TF - IDF

#### Feature

In [79]:
print ("Creating the TF - IDF...\n")
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
tf_idf = TfidfVectorizer(analyzer = "word",   \
                             tokenizer = my_tokenizer,    \
                             preprocessor = None, \
                             strip_accents=None,
                             max_features = 5000,
                             lowercase=False,
                             stop_words = stop_words)                       
                            

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
X_train_tfidf = tf_idf.fit_transform(X_train['post'])

# Numpy arrays are easy to work with, so convert the result to an 
# array
X_train_tfidf = X_train_tfidf.toarray()

Creating the TF - IDF...



In [80]:
X_train_tfidf.shape

(9843, 5000)

In [81]:
# Create vocabulary
X_train_vocab_tfidf = tf_idf.get_feature_names()

In [82]:
# Sum up the counts of each vocabulary word
X_train_dist_tfidf = np.sum(X_train_tfidf, axis=0)

Nous allons afficher 50 mots les plus fréquents :

In [83]:
freq_features_tfidf = DataFrame(zip(X_train_vocab_tfidf, X_train_dist_tfidf))
freq_features_tfidf.rename({0:'Word', 1:'Frequence'}, axis = 1, inplace = True)
freq_features_tfidf.sort_values(['Frequence'], inplace = True, ascending=False)
freq_features_tfidf['Pourcentage'] = round(freq_features_tfidf['Frequence']*100/sum(freq_features_tfidf['Frequence']),2)
freq_features_tfidf[:50]

,Word,Frequence,Pourcentage
1552,file,232.820247,0.55
4140,string,174.427881,0.41
2457,like,168.319796,0.40
1705,function,150.441034,0.36
1756,get,148.346995,0.35
733,code,147.324237,0.35
1157,difference,146.553129,0.35
1795,git,136.792964,0.32
677,class,131.035812,0.31
3453,python,130.852581,0.31


#### Titre uniquement

In [84]:
X_train_title_tfidf = tf_idf.fit_transform(X_train['title_clean'])

# Numpy arrays are easy to work with, so convert the result to an array
X_train_title_tfidf = X_train_title_tfidf.toarray()

In [85]:
X_train_title_tfidf.shape

(9843, 5000)

In [86]:
# Create vocabulary
X_train_title_vocab_tfidf = tf_idf.get_feature_names()

In [87]:
# Sum up the counts of each vocabulary word
X_train_title_dist_tfidf = np.sum(X_train_title_tfidf, axis=0)

In [88]:
freq_features_tfidf_titre = DataFrame(zip(X_train_title_vocab_tfidf, X_train_title_dist_tfidf))
freq_features_tfidf_titre.rename({0:'Word', 1:'Frequence'}, axis = 1, inplace = True)
freq_features_tfidf_titre.sort_values(['Frequence'], inplace = True, ascending=False)
freq_features_tfidf_titre['Pourcentage'] = round(freq_features_tfidf_titre['Frequence']*100/sum(freq_features_tfidf_titre['Frequence']),2)
freq_features_tfidf_titre[:50]

,Word,Frequence,Pourcentage
1624,file,175.054397,0.87
1356,difference,150.991633,0.75
3330,python,149.926372,0.74
2238,java,137.639501,0.68
4077,string,129.960881,0.64
180,android,129.490973,0.64
2249,javascript,128.998628,0.64
1786,git,124.932353,0.62
1747,get,112.628404,0.56
2457,list,87.502937,0.43


#### Cible

In [89]:
print ("Creating the TF - IDF...\n")

y_train_tfidf = tf_idf.fit_transform(y_train_clean)

# Numpy arrays are easy to work with, so convert the result to an 
# array
y_train_tfidf = y_train_tfidf.toarray()

Creating the TF - IDF...



In [90]:
y_train_tfidf.shape

(9843, 4998)

In [91]:
# Create the vocabulary
y_train_vocab_tfidf = tf_idf.get_feature_names()

In [92]:
# Sum up the counts of each vocabulary word
y_train_dist_tfidf = np.sum(y_train_tfidf, axis=0)

Nous allons afficher 50 mots les plus fréquents :

In [93]:
freq_target_tfidf = DataFrame(zip(y_train_vocab_tfidf, y_train_dist_tfidf))
freq_target_tfidf.rename({0:'Word', 1:'Frequence'}, axis = 1, inplace = True)
freq_target_tfidf.sort_values(['Frequence'], inplace = True, ascending=False)
freq_target_tfidf['Pourcentage'] = round(freq_target_tfidf['Frequence']*100/sum(freq_target_tfidf['Frequence']),2)
freq_target_tfidf[:50]

,Word,Frequence,Pourcentage
2404,javascript,364.054803,2.17
3558,python,322.392284,1.93
2377,java,277.392963,1.66
1808,git,257.362231,1.54
118,android,235.036659,1.40
663,c#,199.293870,1.19
2442,jquery,167.070661,1.00
668,c++,157.892208,0.94
2063,html,151.436102,0.90
1045,css,143.726499,0.86


#### Sauvegarder les données

In [94]:
np.save('Data/X_train_tfidf', X_train_tfidf)
np.save('Data/X_train_vocab_tfidf', X_train_vocab_tfidf)
np.save('Data/X_train_dist_tfidf', X_train_dist_tfidf)
np.save('Data/y_train_tfidf', y_train_tfidf)
np.save('Data/y_train_vocab_tfidf', y_train_vocab_tfidf)
np.save('Data/y_train_dist_tfidf', y_train_dist_tfidf)
np.save('Data/X_train_title_tfidf', X_train_title_tfidf)
np.save('Data/X_train_title_vocab_tfidf', X_train_title_vocab_tfidf)
np.save('Data/X_train_title_dist_tfidf', X_train_title_dist_tfidf)

### Word2vect


Afin de créer un vecteur de ots pour chaque document avec word2vect, nous allons modifier notre fonctions de cleaning. Pour plus d'efficacité de l'algorithme word2vect, nous allons séparer le corps de texte en phrases et garder les stop-words. Nous allons également garder les chiffres et la ponctuation.

#### Préparer les données

Voici notre fonction de cleaning modifiée, avec une option pour enlever les stop-words. Par défaut, la fonction n'enlève pas des stop-words.

In [95]:
def post_to_words_w2v( raw_post, remove_stopwords=False ):
    """Function to convert a raw document to a list of words. 
    This cleaning function do not remove the stop words.
    
    Inputs : 
    
    - raw_post : a single string 
    
    Output :
    
    - a single string containing a preprocessed document"""
    
    # 1. Remove HTML
    post_text = BeautifulSoup(raw_post).get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", post_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                                          
    # 
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Lematize
    lemmatizer = WordNetLemmatizer() 

    lems = []

    for word in words:
        word_clean = lemmatizer.lemmatize(word)
        lems.append(word_clean)
    #
    # 5. Join the words back into one string separated by space, 
    # and return the result.
    return(words) 

In [96]:
# Personnaliser la fonction
# Download the punkt tokenizer for sentence splitting
#import nltk.data #!!! Supprimer ???
#nltk.download()  #!!! Supprimer ??? 

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def post_to_sentences( post, tokenizer):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(post.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( post_to_words_w2v( raw_sentence ))
    #
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [97]:
#Clean all data

X_train['text'] = X_train['title'] + ". " + X_train['body']

sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for post in X_train["text"]:
    sentences += post_to_sentences(post, tokenizer)

Parsing sentences from training set


C:\Users\Utilisateur\Anaconda3\lib\site-packages\bs4\__init__.py:272: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


#### Entraîner le modèle

##### !!! Traduire & faire le tuning de paramètres 

With the list of nicely parsed sentences, we're ready to train the model. There are a number of parameter choices that affect the run time and the quality of the final model that is produced. For details on the algorithms below, see the word2vec API documentation as well as the Google documentation. 

    Architecture: Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results.
    Training algorithm: Hierarchical softmax (default) or negative sampling. For us, the default worked well.
    Downsampling of frequent words: The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model.
    Word vector dimensionality: More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.
    Context / window size: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).
    Worker threads: Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.
    Minimum word count: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.

Choosing parameters is not easy, but once we have chosen our parameters, creating a Word2Vec model is straightforward:

In [98]:
# The model is trained on the sample
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

C:\Users\Utilisateur\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2019-11-11 19:08:19,103 : INFO : 'pattern' package not found; tag filters are not available for English
2019-11-11 19:08:19,103 : INFO : collecting all words and their counts
2019-11-11 19:08:19,103 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-11-11 19:08:19,149 : INFO : PROGRESS: at sentence #10000, processed 235705 words, keeping 12950 word types
2019-11-11 19:08:19,196 : INFO : PROGRESS: at sentence #20000, processed 468938 words, keeping 18871 word types
2019-11-11 19:08:19,243 : INFO : PROGRESS: at sentence #30000, processed 705531 words, keeping 23551 word types
2019-11-11 19:08:19,274 : INFO : PROGRESS: at sentence #40000, processed 940375 words, keeping 27755 word types


Training model...


2019-11-11 19:08:19,305 : INFO : collected 30542 word types from a corpus of 1114323 raw words and 47376 sentences
2019-11-11 19:08:19,321 : INFO : Loading a fresh vocabulary
2019-11-11 19:08:19,321 : INFO : min_count=40 retains 2444 unique words (8% of original 30542, drops 28098)
2019-11-11 19:08:19,337 : INFO : min_count=40 leaves 989920 word corpus (88% of original 1114323, drops 124403)
2019-11-11 19:08:19,337 : INFO : deleting the raw counts dictionary of 30542 items
2019-11-11 19:08:19,337 : INFO : sample=0.001 downsamples 57 most-common words
2019-11-11 19:08:19,337 : INFO : downsampling leaves estimated 747465 word corpus (75.5% of prior 989920)
2019-11-11 19:08:19,352 : INFO : estimated required memory for 2444 words and 300 dimensions: 7087600 bytes
2019-11-11 19:08:19,352 : INFO : resetting layer weights
2019-11-11 19:08:19,383 : INFO : training model with 4 workers on 2444 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=10
2019-11-11 19:08:20,13

Le vocabulaire de mots est stocké dans l'attribute model.vocabulary :

In [99]:
print (model.vocabulary)

In [100]:
#!!! Comment utiliser le vocabulaire ??? 
"python" in (model.vocabulary)

TypeError: argument of type 'Word2VecVocab' is not iterable

#### Tester le modèle

Nous allons tester la méthode doesnt_match qui va rechercher un mot qui est différent des autres dans une chaîne de caractères donnée.

In [ ]:
model.doesnt_match("html css div sql windows".split())

In [ ]:
model.doesnt_match("python javascript jupyter anaconda".split())

In [ ]:
model.doesnt_match("git github fetch browser commit".split())

La méthode similar va sortir une liste de mots qui sont les plus proche de l'expression indiquée :

In [ ]:
model.most_similar("python")

In [ ]:
model.most_similar("html")

In [ ]:
model.most_similar("server")

Nous pouvons aussi étudier la similarité entre deux mots :

In [ ]:
model.similarity(w1="python", w2="python")

In [ ]:
model.similarity(w1="python", w2="pip")

In [ ]:
model.similarity(w1="python", w2="windows")

In [ ]:
model.similarity(w1="windows", w2="linux")

#### Représentation numérique de mots

In [ ]:
type(model.syn0_lockf)

In [ ]:
#Number of words in the model
model.syn0_lockf.shape

In [ ]:
#Length of the word vector = num_features (attribute of w2v function) 
len(model.wv["python"])

In [ ]:
#Exemple of the word vector
model.wv["python"][:10]

In [ ]:
# Comment faire ???
"python" in model.vocabulary

In [ ]:
# !!! Dans le tuto, ils utilisent l'attribut model.vocab qui n'existe pas / plus 
# https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-auto-examples-tutorials-run-word2vec-py

for i, word in enumerate(model.vocabulary):
    if i==10:
        break
    print(word)

#### Préparer les vecteurs de mots pour la modélisation

###### !!! Tutorial Kaggle : comprendre la méthode et ré-expliquer 
One challenge with the IMDB dataset is the variable-length reviews. We need to find a way to take individual word vectors and transform them into a feature set that is the same length for every review.

Since each word is a vector in 300-dimensional space, we can use vector operations to combine the words in each review. One method we tried was to simply average the word vectors in a given review (for this purpose, we removed stop words, which would just add noise).

In [ ]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(posts, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    postFeatureVecs = np.zeros((len(posts),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for post in posts:
       #
       # Print a status message every 1000th review
        if counter%1000. == 0.:
            print ("Post %d of %d" % (counter, len(posts)))
       # 
       # Call the function (defined above) that makes average feature vectors
        postFeatureVecs[counter] = makeFeatureVec(post, model, \
           num_features)
       #
       # Increment the counter
        counter = counter + 1.
    return postFeatureVecs

In [ ]:
model.wv.index2word

In [ ]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_posts = []
for post in X_train["text"]:
    clean_train_posts.append( post_to_words_w2v( post, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_posts, model, num_features )

#print "Creating average feature vecs for test review"
#clean_test_posts = []
#for post in test["post"]:
#    clean_test_posts.append( post_to_wordlist( post, \
#        remove_stopwords=True ))

#testDataVecs = getAvgFeatureVecs( clean_test_posts, model, num_features )

# Analyse exploratoire

## Description générale

Nous allons tout d'abord étudier la longueur de notre chaîne de caractère de post, ainsi que le nombre de tags attribué à chaque post :

In [ ]:
nbr_mots = X_train['post'].apply(lambda row : len(row.split()))

In [ ]:
nbr_mots.describe()

Feature post, qui contient le titre, le corps de post et les bigrams créés à partir de titre et le corps de post, contient en moyen 163 mots. Le minimum est 2 mots et le maximum 8678 mots.


In [ ]:
#!!! Améliorer le graphique pour la présentation
plt.hist(nbr_mots, alpha=0.3, range=(
    0, 5600),  bins=200, density=True)
sns.kdeplot(nbr_mots)

plt.xlim(0, 5600)
plt.rcParams['figure.figsize'] = (20, 20)

In [ ]:
y_train_clean.head()

In [ ]:
nbr_tags = y_train_clean.apply(lambda row : len(row.split()))

In [ ]:
nbr_tags.describe()

Le nombre de tags varie de 1 à 6 avec la valeurs médiane de 3 tags par post.

## Fréquences des expressions

### Bag of words

Nous allons afficher les fréquences de 50 features les plus utilisées :

In [ ]:
freq_features_50 = freq_features_bow[:50]
freq_features_50.sort_values(by=['Frequence'], ascending=True, inplace = True)

fig = plt.figure(figsize=(10, 14))

# set params of the graphic
ax = fig.add_subplot(111)
ax.set_xlabel('Fréquence', fontsize=15)
ax.set_title('Les mots les plus fréquents', fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=14)

# set height of bar
bars1 = freq_features_50['Frequence']

# Make the plot
plt.barh(range(50), bars1, color='blue', edgecolor='white')

# Add xticks on the middle of the group bars
plt.yticks(
    range(50), freq_features_50['Word'], fontsize=15)

# Show graphic
plt.show()

Faisons le même affichage pour les tags.

In [ ]:
freq_target_50 = freq_target_bow[:50]
freq_target_50.sort_values(by=['Frequence'], ascending=True, inplace = True)

fig = plt.figure(figsize=(10, 14))

# set params of the graphic
ax = fig.add_subplot(111)
ax.set_xlabel('Fréquence', fontsize=15)
ax.set_title('Les tags les plus fréquents', fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=14)

# set height of bar
bars1 = freq_target_50['Frequence']

# Make the plot
plt.barh(range(50), bars1, color='tomato', edgecolor='white')

# Add xticks on the middle of the group bars
plt.yticks(
    range(50), freq_target_50['Word'], fontsize=15)

# Show graphic
plt.show()

### TF-IDF

Pour pouvoir faire comparaison, nous allons aussi afficher les mot les plus fréquentes en se basant sur les fréquences relatives TF-IDF :

In [ ]:
freq_features_50 = freq_features_tfidf[:50]
freq_features_50.sort_values(by=['Frequence'], ascending=True, inplace = True)

fig = plt.figure(figsize=(10, 14))

# set params of the graphic
ax = fig.add_subplot(111)
ax.set_xlabel('Fréquence', fontsize=15)
ax.set_title('Les mots les plus fréquents TF-IDF', fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=14)

# set height of bar
bars1 = freq_features_50['Frequence']

# Make the plot
plt.barh(range(50), bars1, color='dodgerblue', edgecolor='white')

# Add xticks on the middle of the group bars
plt.yticks(
    range(50), freq_features_50['Word'], fontsize=15)

# Show graphic
plt.show()

Affichons également un graphique de tags :

In [ ]:
freq_target_50 = freq_target_tfidf[:50]
freq_target_50.sort_values(by=['Frequence'], ascending=True, inplace = True)

fig = plt.figure(figsize=(10, 14))

# set params of the graphic
ax = fig.add_subplot(111)
ax.set_xlabel('Fréquence', fontsize=15)
ax.set_title('Les tags les plus fréquents TF-IDF', fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=14)

# set height of bar
bars1 = freq_target_50['Frequence']

# Make the plot
plt.barh(range(50), bars1, color='lightsalmon', edgecolor='white')

# Add xticks on the middle of the group bars
plt.yticks(
    range(50), freq_target_50['Word'], fontsize=15)

# Show graphic
plt.show()

Nous constatons seulement légères différences par rapport à la méthode bag of words.

## Réduction de dimension

### Analyse en composantes principales

Nous allons effectuer une analyse en composante principales afin de visualiser les données. L'ACP va être performée sur les données transformées par TF-IDF.

Nous avons 25000 posts, le calcul de ACP est donc assez exigeant sur une matrice 25 000 x 5 000.

Nous allons utiliser un échantillon de 500 premiers posts qui représentent 2% de données. 

#### Création d'échantillon

In [ ]:
X_train_tfidf_sample = X_train_tfidf[:500]

Afin de pouvoir afficher des tags qui nous intéressent, nous allons aussi échantilloner les tags cleanés correspondants.

In [ ]:
y_train_clean_sample = y_train_clean[:500].reset_index(drop=True)

#### Créer le modèle

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X_train_tfidf_sample)
principalDf = pd.DataFrame(data=principalComponents, columns=[
                           'principal component 1', 'principal component 2'])

In [ ]:
# Scatterplot of all sample posts
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1)
ax.set_xlabel('Principal Component 1', fontsize=10)
ax.set_ylabel('Principal Component 2', fontsize=10)
ax.set_title('2 component PCA, sample of 500 posts using TF-IDF transformation', fontsize=14)

ax.scatter(principalDf['principal component 1'],
           principalDf['principal component 2'], s=50, alpha=0.2)
ax.grid()

####  Affichage de groupes de tags

Nous allons choisir 12 tags à afficher - essayeons de prendre 4 groupes de 3 tags avec des sujets qui sont similaire dans chaque groupe. Afin de trouver la position des tags, nous allons construire une fonction qui renvoi un échantillon de n indexes des expressions qui contient un mot donné. 

In [ ]:
def choose_expression(ser, word, n=3):

    """Function which returns a list of n random index positions of an expression containing indicated string

    Inputs:

    - ser : a serie of expressions
    - word : a string to search for
    - n : number of returned positions (n=3 by default)

    Output:

    - a list of n positions
    """

    index_list = []

    for i in range(len(ser)):
        if word in ser[i]:
            index = i
            index_list.append(index)
        else:
            pass
    
    index_list = random.sample(index_list, n)
    
    return index_list

In [ ]:
# Create the first list of positions containing the word "javascript"
liste_javascript = choose_expression(y_train_clean_sample, 'javascript')
liste_javascript

In [ ]:
# Check
print(y_train_clean_sample[liste_javascript[0]])
print(y_train_clean_sample[liste_javascript[1]])
print(y_train_clean_sample[liste_javascript[2]])

In [ ]:
# Create other lists containing "python", "git" and "android"
liste_python = choose_expression(y_train_clean_sample, 'python')
liste_git = choose_expression(y_train_clean_sample, 'git')
liste_android = choose_expression(y_train_clean_sample, 'android')

In [ ]:
# Create an overall list
chosen_tags_index = liste_javascript + liste_python + liste_git + liste_android

In [ ]:
chosen_tags_index

In [ ]:
# View the chosen tags
chosen_tags = y_train_clean_sample[chosen_tags_index] 
chosen_tags

In [ ]:
chosen_principalDf = principalDf.iloc[chosen_tags_index]
chosen_principalDf

In [ ]:
# Scatterplot of chosen tags
# !!! Comment faire pour éviter le texte qui se chévauche ?

sns.set()# Initialize figure
fig, ax = plt.subplots(figsize = (15, 15))

ax.set_title('2 component PCA using TF-IDF transformation, chosen tags', fontsize=14)

sns.scatterplot(chosen_principalDf['principal component 1'],
           chosen_principalDf['principal component 2'], alpha = 0.5)
 
for i in chosen_tags.index:    
    
    plt.text(chosen_principalDf['principal component 1'][i], chosen_principalDf['principal component 2'][i], 
             chosen_tags[i],fontsize = 14)

plt.show()

Nous pouvons voir que les point représentants des posts sont proches selon le groupe - les tags contenant "python" sont en haut à gauche, "git"s sont en haut à droite, "javascript" se mélange un peu avec "android", mais les groupes sont quand même séparables. 

### t-SNE 

#### Créer le modèle

In [ ]:
# Initialize t-SNE
tsne = TSNE(n_components = 2, init = 'random', random_state = 10, perplexity = 100)

In [ ]:
tsne_df = tsne.fit_transform(X_train_tfidf_sample)

In [ ]:
principalDf = pd.DataFrame(data=tsne_df, columns=[
                           'principal component 1', 'principal component 2'])

In [ ]:
# Scatterplot of all sample posts
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1)
ax.set_xlabel('Principal Component 1', fontsize=10)
ax.set_ylabel('Principal Component 2', fontsize=10)
ax.set_title('2 component t-SNE, sample of 500 posts using TF-IDF transformation', fontsize=14)

ax.scatter(principalDf['principal component 1'],
           principalDf['principal component 2'], s=50, alpha=0.5)
ax.grid()

Le nuage de points créé à l'aide de t-SNE semble former un rond.

#### Affichage de groupes de tags

In [ ]:
chosen_principalDf = principalDf.iloc[chosen_tags_index]

In [ ]:
# Scatterplot of chosen tags

sns.set()# Initialize figure
fig, ax = plt.subplots(figsize = (15, 15))

ax.set_title('2 component t-SNE using TF-IDF transformation, chosen tags', fontsize=14)

sns.scatterplot(chosen_principalDf['principal component 1'],
           chosen_principalDf['principal component 2'], alpha = 0.5)
 
for i in chosen_tags.index:    
    
    plt.text(chosen_principalDf['principal component 1'][i], chosen_principalDf['principal component 2'][i], 
             chosen_tags[i],fontsize = 14)

plt.show()

Nous pouvons trouver certains tags avec la même thématique proche les uns des autres (git, python), mais la séparation n'est pas très prononcée.